In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# 제품 데이터 저장 리스트
products_data = []

# Selenium 드라이버 설정
def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    # Selenium 탐지 우회
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# 제품 정보 추출 함수
def extract_product_info(product, driver):
    try:
        if product.get("id", "").startswith("adPointProductItem") or "adSmartAreaTop" in product.get("id", ""):
            return False

        photo_elem = product.select_one(".thumb_image img")
        if photo_elem:
            photo = photo_elem.get("data-original") or photo_elem.get("src") or "N/A"
            if "noImg_130.gif" in photo:
                photo = "N/A"
        else:
            photo = "N/A"

        name_elem = product.select_one(".prod_name a")
        name = name_elem.text.strip() if name_elem else "Unknown"
        detail_url = name_elem["href"] if name_elem and "href" in name_elem.attrs else None

        # 상세 페이지에서 spec 수집
        spec_dict = {}
        if detail_url:
            # 현재 탭의 핸들 저장
            main_window = driver.current_window_handle

            # 새 탭 열기 (JavaScript 실행)
            driver.execute_script("window.open('', '_blank');")

            # 새 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            # 새 탭에서 상세 페이지 열기
            driver.get(detail_url)
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_spec table"))
                )
            except:
                print(f"❗️스펙 테이블 로딩 실패 (대기 시간 초과): {name}")

            detail_soup = BeautifulSoup(driver.page_source, "html.parser")

            spec_table = detail_soup.select_one(".prod_spec .spec_tbl tbody") or detail_soup.select_one(".prod_spec table")
            if spec_table:
                for row in spec_table.find_all("tr"):
                    keys = row.find_all("th", class_="tit")
                    values = row.find_all("td", class_="dsc")
                    for key_elem, value_elem in zip(keys, values):
                        key = key_elem.get_text(strip=True)
                        value = value_elem.get_text(strip=True)
                        spec_dict[key] = value

            # 탭 닫기
            driver.close()

            # 원래 탭으로 돌아가기
            driver.switch_to.window(main_window)

        price_elem = product.select_one(".price_sect a")
        price = price_elem.text.strip().replace(",", "") if price_elem else "N/A"

        rating_elem = product.select_one(".star-single .text__score") or product.select_one(".point_num")
        rating = rating_elem.text.strip() if rating_elem else "N/A"

        product_info = {
            "url": photo,
            "Name": name,
            "Price": price,
            "Rating": rating,
            **spec_dict
        }
        products_data.append(product_info)
        print(f"수집: {name}, 사진: {photo}, 가격: {price}, 평점: {rating}")
        return True

    except Exception as e:
        print(f"제품 정보 수집 오류: {e}")

        # 혹시 새 탭이 열려있는 상태라면 닫고 메인 탭으로 돌아가기
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass

        return False

# 페이지 스크레이핑 함수
def scrape_page(driver, page_num):
    try:
        # 현재 페이지의 제품 정보 수집
        soup = BeautifulSoup(driver.page_source, "html.parser")
        products = soup.select(".prod_item.prod_layer")
        print(f"페이지 {page_num}에서 찾은 제품 수: {len(products)}")

        if not products:
            print("더 이상 제품이 없습니다.")
            return False

        for product in products:
            if product.find_parent(id="adPointArea") or product.find_parent(id="adSmartAreaTop"):
                continue
            extract_product_info(product, driver)

        return True
    except Exception as e:
        print(f"페이지 수집 오류 (페이지 {page_num}): {e}")
        return False

# 크롤링 함수
def crawl_danawa_cpu_to_dataframe(max_pages=None):
    driver = setup_driver()
    try:
        base_url = "https://prod.danawa.com/list/?cate=112753"
        driver.get(base_url)
        time.sleep(3)  # 초기 페이지 로딩 대기

        page_num = 1
        while True:
            if max_pages and page_num > max_pages:
                break

            print(f"페이지 {page_num} 수집 중...")

            # 현재 페이지 수집
            scrape_page(driver, page_num)

            if max_pages and page_num >= max_pages:
                break

            # 다음 페이지로 이동
            try:
                # 페이지 이동 방식 결정
                if page_num % 10 == 0:  # 10, 20, 30 등 10의 배수 페이지일 경우
                    # 다음 페이지 그룹 버튼 클릭
                    next_page_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.edge_nav.nav_next"))
                    )
                    next_page_button.click()
                else:
                    # 일반 페이지 이동
                    driver.execute_script(f"movePage({page_num + 1}); return false;")

                # 페이지 로딩 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_item.prod_layer"))
                )
                time.sleep(2)  # 추가 대기 시간

                page_num += 1
            except Exception as e:
                print(f"다음 페이지로 이동 실패: {e}")
                break

        df = pd.DataFrame(products_data)
        print(f"총 {len(products_data)}개 CPU 제품 수집 완료")
        return df

    finally:
        driver.quit()
        print("Selenium 드라이버 종료")

# 실행
max_pages = 35  # 테스트를 위해 작은 값으로 설정
df = crawl_danawa_cpu_to_dataframe(max_pages=max_pages)
print("\n수집된 데이터프레임 미리보기:")
print(df.head())
df.to_csv("danawa_gpu_products.csv", index=False, encoding="utf-8-sig")
print("데이터가 'danawa_gpu_products.csv' 파일로 저장되었습니다.")

페이지 1 수집 중...
페이지 1에서 찾은 제품 수: 30
수집: ASRock 라데온 RX 9070 XT 스틸레전드 D6 16GB 대원씨티에스, 사진: //img.danawa.com/prod_img/500000/285/318/img/77318285_1.jpg?shrink=130:130&_v=20250306230129, 가격: 1213920원, 평점: N/A
수집: 이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB, 사진: //img.danawa.com/prod_img/500000/939/161/img/27161939_1.jpg?shrink=130:130&_v=20240308094830, 가격: 487000원, 평점: 4.8
수집: 갤럭시 GALAX 지포스 RTX 5080 WHITE OC D7 16GB, 사진: //img.danawa.com/prod_img/500000/386/075/img/75075386_1.jpg?shrink=130:130&_v=20250204101702, 가격: 1961990원, 평점: 5.0
수집: ZOTAC GAMING 지포스 RTX 5070 SOLID OC D7 12GB, 사진: //img.danawa.com/prod_img/500000/413/319/img/77319413_1.jpg?shrink=130:130&_v=20250305235713, 가격: 988030원, 평점: 4.7
수집: MSI 지포스 RTX 5080 뱅가드 SOC D7 16GB 하이퍼프로져, 사진: //img.danawa.com/prod_img/500000/700/541/img/75541700_1.jpg?shrink=130:130&_v=20250318183451, 가격: 2248920원, 평점: 5.0
수집: PALIT 지포스 RTX 5070 Ti GAMINGPRO D7 16GB 이엠텍, 사진: //img.danawa.com/prod_img/500000/952/465/img/76465952_1.jpg?shrink=130:130&_v=202504

In [4]:
import pandas as pd
df = pd.read_csv("danawa_gpu_products.csv", encoding="UTF-8")
df

,url,Name,Price,Rating,제조회사,등록년월,칩셋 제조사,제품 시리즈,GPU 제조 공정,AMD 칩셋,...,DP,USB Type-C,기가비트LAN,세로(높이),그래픽카드 구성,12K 지원,썬더볼트3,USB3.x 5Gbps,파워 구성,기타 칩셋
0,//img.danawa.com/prod_img/500000/285/318/img/7...,ASRock 라데온 RX 9070 XT 스틸레전드 D6 16GB 대원씨티에스,1213920원,NaN,ASRock(제조사 웹사이트 바로가기),2025년 03월,AMD(ATi),라데온 RX 9000,4nm,RX 9070 XT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,//img.danawa.com/prod_img/500000/939/161/img/2...,이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB,487000원,4.8,이엠텍(제조사 웹사이트 바로가기),2023년 08월,NVIDIA,지포스 RTX 40,4nm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,//img.danawa.com/prod_img/500000/386/075/img/7...,갤럭시 GALAX 지포스 RTX 5080 WHITE OC D7 16GB,1961990원,5.0,갤럭시(제조사 웹사이트 바로가기),2025년 01월,NVIDIA,지포스 RTX 50,TSMC 4nm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,//img.danawa.com/prod_img/500000/413/319/img/7...,ZOTAC GAMING 지포스 RTX 5070 SOLID OC D7 12GB,988030원,4.7,ZOTAC(제조사 웹사이트 바로가기),2025년 03월,NVIDIA,지포스 RTX 50,4nm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,//img.danawa.com/prod_img/500000/700/541/img/7...,MSI 지포스 RTX 5080 뱅가드 SOC D7 16GB 하이퍼프로져,2248920원,5.0,MSI(제조사 웹사이트 바로가기),2025년 02월,NVIDIA,지포스 RTX 50,TSMC 4nm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,//img.danawa.com/prod_img/500000/912/159/img/1...,이도디스플레이 지포스 GTX 1650 RAGE-X D6 4GB Dual,244000원,5.0,이도디스플레이(제조사 웹사이트 바로가기),2022년 05월,NVIDIA,지포스 16,12nm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965,//img.danawa.com/prod_img/500000/385/196/img/1...,NVIDIA RTX A4000 D6 16GB (해외구매),1819000원,NaN,NVIDIA(제조사 웹사이트 바로가기),2021년 08월,NVIDIA,RTX A,NaN,NaN,...,NaN,NaN,NaN,110mm,NaN,NaN,NaN,NaN,NaN,NaN
966,//img.danawa.com/prod_img/500000/312/474/img/1...,ASUS TUF Gaming 지포스 GTX 1660 SUPER O6G D6 6GB ...,450030원,NaN,ASUS(제조사 웹사이트 바로가기),2019년 11월,NVIDIA,지포스 16,NaN,NaN,...,NaN,NaN,NaN,124mm,NaN,NaN,NaN,NaN,NaN,NaN
967,//img.danawa.com/prod_img/500000/886/473/img/1...,MSI 지포스 RTX 3060 게이밍 X D6 12GB 트윈프로져8 (해외구매),509600원,4.0,MSI(제조사 웹사이트 바로가기),2021년 02월,NVIDIA,지포스 RTX 30,NaN,NaN,...,NaN,NaN,NaN,131mm,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# c 컬럼의 인덱스 찾기
index = df.columns.get_loc('적합성평가인증')

# c 컬럼 좌측의 컬럼들 선택
left_columns = df.columns[:index]
df = df[left_columns]

# 공백을 기준으로 첫 번째 단어 추출
df['등록년'] = df['등록년월'].str.split().str[0]
df['등록월'] = df['등록년월'].str.split().str[1]
df['제조회사'] = df['제조회사'].str.split('\(').str[0]

df.drop(['등록년월', 'Unnamed: 23'], axis=1, inplace=True)

df

<>:11: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\('
C:\Users\ms0\AppData\Local\Temp\ipykernel_2116\3903293572.py:11: SyntaxWarning: invalid escape sequence '\('
  df['제조회사'] = df['제조회사'].str.split('\(').str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_2116\3903293572.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['등록년'] = df['등록년월'].str.split().str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_2116\3903293572.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,url,Name,Price,Rating,제조회사,칩셋 제조사,제품 시리즈,GPU 제조 공정,AMD 칩셋,인터페이스,...,백플레이트,DrMOS,LED 라이트,LED온오프스위치,측면 LED,팬 LED,VGA지지대,3x8핀 to 16핀 커넥터,등록년,등록월
0,//img.danawa.com/prod_img/500000/285/318/img/7...,ASRock 라데온 RX 9070 XT 스틸레전드 D6 16GB 대원씨티에스,1213920원,NaN,ASRock,AMD(ATi),라데온 RX 9000,4nm,RX 9070 XT,PCIe5.0x16,...,○,○,○,○,○,○,○,○,2025년,03월
1,//img.danawa.com/prod_img/500000/939/161/img/2...,이엠텍 지포스 RTX 4060 STORM X Dual OC D6 8GB,487000원,4.8,이엠텍,NVIDIA,지포스 RTX 40,4nm,NaN,PCIe4.0x16(at x8),...,○,NaN,○,NaN,○,NaN,NaN,NaN,2023년,08월
2,//img.danawa.com/prod_img/500000/386/075/img/7...,갤럭시 GALAX 지포스 RTX 5080 WHITE OC D7 16GB,1961990원,5.0,갤럭시,NVIDIA,지포스 RTX 50,TSMC 4nm,NaN,PCIe5.0x16,...,○,NaN,○,NaN,○,○,○,○,2025년,01월
3,//img.danawa.com/prod_img/500000/413/319/img/7...,ZOTAC GAMING 지포스 RTX 5070 SOLID OC D7 12GB,988030원,4.7,ZOTAC,NVIDIA,지포스 RTX 50,4nm,NaN,PCIe5.0x16,...,○,NaN,○,NaN,NaN,NaN,○,NaN,2025년,03월
4,//img.danawa.com/prod_img/500000/700/541/img/7...,MSI 지포스 RTX 5080 뱅가드 SOC D7 16GB 하이퍼프로져,2248920원,5.0,MSI,NVIDIA,지포스 RTX 50,TSMC 4nm,NaN,PCIe5.0x16,...,○,○,○,NaN,○,NaN,○,○,2025년,02월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,//img.danawa.com/prod_img/500000/912/159/img/1...,이도디스플레이 지포스 GTX 1650 RAGE-X D6 4GB Dual,244000원,5.0,이도디스플레이,NVIDIA,지포스 16,12nm,NaN,PCIe3.0x16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022년,05월
965,//img.danawa.com/prod_img/500000/385/196/img/1...,NVIDIA RTX A4000 D6 16GB (해외구매),1819000원,NaN,NVIDIA,NVIDIA,RTX A,NaN,NaN,PCIe4.0x16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021년,08월
966,//img.danawa.com/prod_img/500000/312/474/img/1...,ASUS TUF Gaming 지포스 GTX 1660 SUPER O6G D6 6GB ...,450030원,NaN,ASUS,NVIDIA,지포스 16,NaN,NaN,PCIe3.0x16,...,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019년,11월
967,//img.danawa.com/prod_img/500000/886/473/img/1...,MSI 지포스 RTX 3060 게이밍 X D6 12GB 트윈프로져8 (해외구매),509600원,4.0,MSI,NVIDIA,지포스 RTX 30,NaN,NaN,PCIe4.0x16,...,○,NaN,○,NaN,○,NaN,NaN,NaN,2021년,02월


In [3]:
df.to_csv("gpu.csv", index=False, encoding="utf-8-sig")

In [3]:
df.head().to_csv("gpu_sample.csv", index=False, encoding="utf-8-sig")

In [13]:
# 1. 첫 번째 테이블: idx (1000부터 시작)와 a, b 컬럼
gpu_product_df = pd.DataFrame({
    'product_idx': range(535, 535 + len(df)),
    'name': df['Name'],
    'brand': df['제조회사'],
    'price': df['Price'],
    'rating': df['Rating'],
    'category': "GPU"
})

# 2. 두 번째 테이블: idx (index+1), table1의 idx를 외래키로, c 컬럼
gpu_img_df = pd.DataFrame({
    'product_image_idx': range(535, 535 + len(df)),
    'product_idx': gpu_product_df['product_idx'],  # table1의 idx를 외래키로
    'img_url': df['url']
})

used_columns = ['Name', 'Price', 'url', 'Rating', '제조회사']
# 원래 데이터프레임에서 사용되지 않은 컬럼 선택
remaining_columns = [col for col in df.columns if col not in used_columns]

# table3 생성
gpu_detail_df = pd.DataFrame({
    'gpu_idx': range(1, len(df) + 1),
    'product_idx': gpu_product_df['product_idx']
})
# 나머지 컬럼 추가
for col in remaining_columns:
    gpu_detail_df[col] = df[col]

In [14]:
gpu_img_df.to_csv("gpu_img.csv", index=False, encoding="utf-8-sig")

In [15]:
gpu_product_df.to_csv("gpu_product.csv", index=False, encoding="utf-8-sig")

In [21]:
gpu_detail_df.to_csv("gpu_detail.csv", index=False, encoding="utf-8-sig")

In [19]:
gpu_detail_df.drop(['AMD 칩셋', '8K 지원', 'HDR 지원', 'HDCP 지원', '방열판','히트파이프', '팬쿨러','제로팬(0-dB기술)','백플레이트', 'DrMOS', 'LED 라이트', 'LED온오프스위치', '측면 LED', '팬 LED', 'VGA지지대', '3x8핀 to 16핀 커넥터'], axis=1, inplace=True)

In [23]:
gpu_detail_df

,gpu_idx,product_idx,칩셋 제조사,제품 시리즈,GPU 제조 공정,인터페이스,권장 파워용량,전원 포트,가로(길이),부스트클럭,...,메모리 클럭,메모리 용량,메모리 버스,HDMI,DisplayPort,모니터 지원,팬 개수,두께,등록년,등록월
0,1,535,AMD(ATi),라데온 RX 9000,4nm,PCIe5.0x16,정격파워800W 이상,8핀 x2,298mm,2970MHz,...,20000MHz,16GB,256-bit,1개,3개,최대 모니터 4개,3팬,58mm,2025년,03월
1,2,536,NVIDIA,지포스 RTX 40,4nm,PCIe4.0x16(at x8),정격파워600W 이상,8핀 x1,249.9mm,2670MHz,...,17000MHz,8GB,128-bit,1개,3개,최대 모니터 4개,2팬,40.2mm,2023년,08월
2,3,537,NVIDIA,지포스 RTX 50,TSMC 4nm,PCIe5.0x16,정격파워850W 이상,16핀(12V2x6) x1,303mm,2625MHz,...,30000MHz,16GB,256-bit,1개,3개,최대 모니터 4개,3팬,49.7mm,2025년,01월
3,4,538,NVIDIA,지포스 RTX 50,4nm,PCIe5.0x16,정격파워650W 이상,16핀(12V2x6) x1,304.4mm,2542MHz,...,28000MHz,12GB,192-bit,1개,3개,최대 모니터 4개,3팬,41.6mm,2025년,03월
4,5,539,NVIDIA,지포스 RTX 50,TSMC 4nm,PCIe5.0x16,정격파워850W 이상,16핀(12V2x6) x1,357mm,2730MHz,...,30000MHz,16GB,256-bit,1개,3개,최대 모니터 4개,3팬,66mm,2025년,02월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,965,1499,NVIDIA,지포스 16,12nm,PCIe3.0x16,정격파워300W 이상,6핀 x1,250mm,1590MHz,...,12000MHz,4GB,128-bit,1개,1개,최대 모니터 3개,2팬,41mm,2022년,05월
965,966,1500,NVIDIA,RTX A,NaN,PCIe4.0x16,NaN,NaN,240mm,NaN,...,16000MHz,16GB,256-bit,NaN,4개,최대 모니터 4개,1팬,20mm,2021년,08월
966,967,1501,NVIDIA,지포스 16,NaN,PCIe3.0x16,정격파워450W 이상,NaN,206mm,1845MHz,...,14002MHz,6GB,192-bit,NaN,1개,최대 모니터 3개,2팬,46mm,2019년,11월
967,968,1502,NVIDIA,지포스 RTX 30,NaN,PCIe4.0x16,정격파워550W 이상,NaN,276mm,1837MHz,...,15000MHz,12GB,192-bit,NaN,3개,최대 모니터 4개,2팬,51mm,2021년,02월
